<a href="https://colab.research.google.com/github/bdub99/Big-Data-ETL/blob/main/part_one_tools_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-01-04 03:07:21--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.73MB/s    in 0.2s    

2023-01-04 03:07:21 (5.73 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz" 
spark.sparkContext.addFile(url)
tools_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Tools_v1_00.tsv.gz"), sep='\t', header=True)

# Show DataFrame
tools_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

In [5]:
# Get the number of rows in the DataFrame.
tools_df.count()

1741100

# Transform the Data

## Create the "review_id_table".

In [6]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_table = tools_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2UM5QMHBHC90Q|   15785389|B00H5U9ZD6|     115362950| 2015-08-31|
|  RF0D1LEIF6L7|   47910848|B001TJGCS0|     570955425| 2015-08-31|
|  RM6YKIWQVNSY|   36328996|B000NIK8JW|     128843593| 2015-08-31|
|R1RL3L68ASPS36|   51785809|B008ZYRGUA|     407828107| 2015-08-31|
|R1U4XFBFAG34CY|   40757491|B00K5CA0GC|     490746675| 2015-08-31|
|R3KFIK8P0I91PL|   35544833|B00AIJAA94|     148352067| 2015-08-31|
| RENOAY76PPK1O|   16474909|B00JKEI6K8|     331801084| 2015-08-31|
| RINV884I0NL5V|   22601598|B00AGCHVS6|     471514859| 2015-08-31|
| R5KJH6CXZH2PX|   16129808|B002S0O7U4|     162253576| 2015-08-31|
| RO69JF6QWD0W1|   24382335|B0084YHXMW|      69530650| 2015-08-31|
|R3L9NQBH3FI55C|   49796324|B00MLSS1SW|     916693555| 2015-08-31|
| R4YH95YPHVU0C|   33289687|B00D4WLS2A|      39333316| 2015-08

## Create the "products" Table

In [7]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df_table = tools_df.select(["product_id", "product_title"])
products_df_table.show(20)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00H5U9ZD6|WallPeg 12 sq ft ...|
|B001TJGCS0|Nite Ize Nite Daw...|
|B000NIK8JW|Stanley 84-058 4 ...|
|B008ZYRGUA|Powerextra 14.4V ...|
|B00K5CA0GC|Waterproof Invisi...|
|B00AIJAA94|Crime Scene Do No...|
|B00JKEI6K8|Aweek® 2 Pcs Bicy...|
|B00AGCHVS6|Ryobi P102 Genuin...|
|B002S0O7U4|Wiha 66995 6-Piec...|
|B0084YHXMW|TOMTOP LED Submar...|
|B00MLSS1SW|Black & Decker BD...|
|B00D4WLS2A|Crain 126 Staple ...|
|B00JGCDV5Y|Diamond Semi Roun...|
|B00N0PS3YM|It Mall 9 LED 375...|
|B0000DD4KV|IRWIN Tools Metri...|
|B0009H5FB8|743022-A Backing ...|
|B00NKSMPZW|ClearArmor 141001...|
|B00RBAB92K|KKmoon 9cm Mini A...|
|B00QGBNZVI|Refun E6 High Pow...|
|B00XXU3CDG|Dr.meter S20 Mois...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [8]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = tools_df.groupBy('customer_id').count()
customers_table = customers_df.withColumnRenamed('count', 'customer_count')
customers_table.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   15550969|             2|
|   37757447|             4|
|   13982277|             1|
|   48428870|            10|
|   49606056|             1|
|   35041028|             1|
|   11818958|             1|
|   43222407|             1|
|   42900950|             1|
|   44517006|             1|
|   12084227|             1|
|    2174886|             1|
|   47577568|             1|
|   26073975|             1|
|   51134448|             2|
|    7337298|             2|
|   51068388|             2|
|   36317952|             2|
|   48325001|             1|
|    1619706|             2|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [9]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_table = tools_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(20)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2UM5QMHBHC90Q|          5|            0|          0|   N|
|  RF0D1LEIF6L7|          4|            0|          0|   N|
|  RM6YKIWQVNSY|          1|            6|          6|   N|
|R1RL3L68ASPS36|          4|            0|          0|   N|
|R1U4XFBFAG34CY|          5|            0|          0|   N|
|R3KFIK8P0I91PL|          5|            0|          0|   N|
| RENOAY76PPK1O|          5|            0|          0|   N|
| RINV884I0NL5V|          1|            0|          0|   N|
| R5KJH6CXZH2PX|          5|            0|          0|   N|
| RO69JF6QWD0W1|          5|            0|          0|   N|
|R3L9NQBH3FI55C|          4|            1|          1|   N|
| R4YH95YPHVU0C|          5|            0|          0|   N|
|R10M1WWDDQBGG2|          2|            0|          0|   N|
| RV3KWQBTNIO62|          5|            

# Load

In [10]:
mode = "append"
jdbc_url="jdbc:postgresql://module-22-database.ckochbxtlyig.us-west-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", "password": "postgres", "driver":"org.postgresql.Driver"}

In [11]:
# Write review_id_df to table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [12]:
# Write products_df to table in RDS
products_df_table.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

Py4JJavaError: ignored

In [13]:
# Write customers_df to table in RDS
customers_table.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

Py4JJavaError: ignored

In [14]:
# Write vine_df to table in RDS
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

Py4JJavaError: ignored